# NetColoc analysis of rare variants causing childhood dementia

Following the example protocol here: https://github.com/ucsd-ccbb/NetColoc/blob/d97f2597b6bf73f201a34c15dbba63eac014a793/example_notebooks/ASD_NetColoc_analysis.ipynb 

In [3]:
# load required packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import re
import random

from IPython.display import display

import getpass
import ndex2

import json
import cdapsutil

from gprofiler import GProfiler
gp = GProfiler("MyToolName/0.1")

from scipy.stats import hypergeom
from scipy.stats import norm

# latex rendering of text in graphs
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

sns.set(font_scale=1.4)

sns.set_style('white')

sns.set_style("ticks", {"xtick.major.size": 15, "ytick.major.size": 15})
plt.rcParams['svg.fonttype'] = 'none'

from datetime import datetime
import sys
%matplotlib inline

In [11]:
# verify DDOT was installed
import ddot

from netcoloc import netprop_zscore, netprop, network_colocalization, validation

# 2. Select one gene set of interest. Load gene set from text files into python.

Identify one gene sets of interest. Gene sets should come from experimental data (not manual curation) to avoid bias.

Usage Note: gene sets should be < 500 genes (propagation algorithm breaks down if seeded with larger sets). If your gene set is larger, only use the top 500 as seeds to the network propagation.

Here, will use the childhood dementia genes

In [8]:
# set names of geneset 1
# ------ customize this section based on your gene sets and how they should be labeled -------
d1_name='CD'

In [16]:
# load childhoood dementia genes frmo knowedgebase 
D1_df = pd.read_csv('genesets/childhooddementiagenes.csv')
D1_df.index = D1_df['GENE']
print('Number of '+d1_name+' genes:', len(D1_df))
D1_genes = D1_df.index.tolist() # define rare variant genes to seed network propagation
print("First 5 genes:", D1_genes[0:5])

Number of CD genes: 235
First 5 genes: ['SGSH', 'NAGLU', 'HGSNAT', 'GNS', 'MTTP']


# 3. Select gene interaction network to use for the analysis.

Identify network UUID on NDEx (ndexbio.org) and use this to import to this Jupyter notebook. We recommend using PCNet as a starting point (will try this now), but a user may want to switch to “STRING high confidence” if using a machine with low memory (< 8GB RAM).

This takes about 2 mins

In [17]:
interactome_uuid='4de852d9-9908-11e9-bcaf-0ac135e8bacf' # for PCNet
# interactome_uuid='275bd84e-3d18-11e8-a935-0ac135e8bacf' # for STRING high confidence
ndex_server='public.ndexbio.org'
ndex_user=None
ndex_password=None
G_int = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid=interactome_uuid
        ).to_networkx()
nodes = list(G_int.nodes)

# remove self edges from network
G_int.remove_edges_from(nx.selfloop_edges(G_int))

# print out the numbers of nodes and edges in the interatome for diagnostic purposes:
print('Number of nodes:', len(G_int.nodes))
print('\nNumber of edges:', len(G_int.edges))

Number of nodes: 18820

Number of edges: 2693109


In [18]:
int_nodes = list(G_int.nodes)

# Identify network colocalized gene network
## 4. Precalculate matrices needed for propagation. This step should take a few minutes (more for larger/denser networks)
A benchmarking analysis demonstrates that the runtime required scales with the number of edges (w’) and the number of nodes (w’’). NetColoc includes functionality for saving and loading these matrices, by setting the XXX parameter, which can be useful if running multiple analyses. The diffusion parameter, which controls the rate of propagation through the network, may be set in this step. In practice, we have found that results are not dependent on the choice of this parameter, and recommend using the default value of 0.5.

Background on network propagation: https://www.nature.com/articles/nrg.2017.38.pdf?origin=ppub

In [19]:
# pre-calculate matrices used for network propagation. this step takes a few minutes, more for denser interactomes
print('\ncalculating w_prime')
w_prime = netprop.get_normalized_adjacency_matrix(G_int, conserve_heat=True)

print('\ncalculating w_double_prime')
w_double_prime = netprop.get_individual_heats_matrix(w_prime, .5)


calculating w_prime

calculating w_double_prime


# 5. Subset seed genes to those found in the selected network.

Only genes contained in the interaction network will be retained for downstream analysis.

In [20]:
# subset seed genes to those found in interactome
print("Number of D1 genes:", len(D1_genes))
D1_genes = list(np.intersect1d(D1_genes,int_nodes))
print("Number of D1 genes in interactome:", len(D1_genes))

Number of D1 genes: 235
Number of D1 genes in interactome: 196


# 6. Compute network proximity scores from seed gene set.

The network proximity scores include a correction for the degree distribution of the input gene sets. The runtime required for computing the network proximity scores increases linearly with the number of nodes in the underlying interaction network and with the size of the input gene list

In [21]:
# D1 network propagation
print('\nCalculating D1 z-scores: ')
z_D1, Fnew_D1, Fnew_rand_D1 = netprop_zscore.calculate_heat_zscores(w_double_prime, int_nodes, 
                                                                    dict(G_int.degree), 
                                                                    D1_genes, num_reps=1000,
                                                                    minimum_bin_size=100)

z_D1 = pd.DataFrame({'z':z_D1})

z_D1.sort_values('z',ascending=False).head()


Calculating D1 z-scores: 


  0%|          | 0/1000 [00:00<?, ?it/s]

,z
DLAT,23.819753
EIF2B1,23.796715
LRPPRC,23.454761
PDHB,23.177739
DLD,22.877058


# 7. Build proximal subnetwork by taking z> thresh

Select genes which have z>threshold (default = 3). These genes are proximal in network space to the seed gene set

In [22]:
# ----------- select thresholds for NetColoc -----------------
zthresh=3 # default = 3

# select the genes in the network intersection, make a subgraph

G_prox = nx.subgraph(G_int,z_D1[z_D1['z']>zthresh].index.tolist()) 
print("Nodes in proximal subgraph:", len(G_prox.nodes()))
print("Edges in proximal subgraph:", len(G_prox.edges()))

Nodes in proximal subgraph: 748
Edges in proximal subgraph: 31926


# Compute systems map from proximal subgraph

9. Convert network colocalization subnetwork to form used in community detection module

In [23]:
# compile dataframe of metadata for overlapping nodes
node_df = pd.DataFrame(index=list(G_prox.nodes))
node_df = node_df.assign(d1_seeds=0, d1_name=d1_name,)
node_df.loc[list(np.intersect1d(D1_genes,node_df.index.tolist())), 'd1_seeds']=1
node_df['z_d1']=z_D1.loc[list(G_prox.nodes)]['z']
node_df['sum_seeds']=node_df['d1_seeds']

node_df = node_df.sort_values('z_d1',ascending=False)
node_df.head(15)

,d1_seeds,d1_name,z_d1,sum_seeds
DLAT,1,CD,23.819753,1
EIF2B1,1,CD,23.796715,1
LRPPRC,1,CD,23.454761,1
PDHB,1,CD,23.177739,1
DLD,1,CD,22.877058,1
EIF2B3,1,CD,22.639214,1
PDHA1,1,CD,22.497164,1
EIF2B5,1,CD,21.419588,1
MTR,1,CD,21.417352,1
DPM1,1,CD,21.376330,1


# 10. Run community detection on NetColoc subnetwork (recommend HiDef).
Documentation for CDAPS utils to build multiscale systems map in notebook

https://cdapsutil.readthedocs.io/en/latest/quicktutorial.html#example
https://cdapsutil.readthedocs.io/en/latest/cdapsutil.html#community-detection

In [24]:
print("Nodes in overlap subgraph:", len(G_prox.nodes()))
print("Edges in overlap subgraph:", len(G_prox.edges()))
# Create cx format of overlap subgraph
G_prox_cx = ndex2.create_nice_cx_from_networkx(G_prox)
G_prox_cx.set_name(d1_name+'_NetColoc_subgraph') 
for node_id, node in G_prox_cx.get_nodes():
    data = node_df.loc[node['n']]
    for row, value in data.items():
        if row == 'd1_seeds' or row == 'd2_seeds' or row=='sum_seeds':
            data_type = 'double'
        elif row=='d1_name' or row=='d2_name':
            data_type='string'
        else:
            data_type = 'double'
        G_prox_cx.set_node_attribute(node_id, row, value, type=data_type)

cd = cdapsutil.CommunityDetection()

# Run HiDeF on CDAPS REST service
G_hier = cd.run_community_detection(G_prox_cx, algorithm='hidefv1.1beta',arguments={'--maxres':'20'})


Nodes in overlap subgraph: 748
Edges in overlap subgraph: 31926


In [25]:
# Print information about hierarchy
print('Hierarchy name: ' + str(G_hier.get_name()))
print('# nodes: ' + str(len(G_hier.get_nodes())))
print('# edges: ' + str(len(G_hier.get_edges())))

Hierarchy name: hidefv1.1beta_(none)_CD_NetColoc_subgraph
# nodes: 69
# edges: 69


11. Convert the NetColoc hierarchy to networkx format, and write out features of the hierarchy to a pandas dataframe, for easier access in Python.

In [26]:
G_hier = G_hier.to_networkx(mode='default')
G_hier

nodes = G_hier.nodes()

# print the number of nodes and edges in the hierarchy for diagnostic purposes
print('Number of nodes:', len(G_hier.nodes()))

print('\nNumber of edges:', len(G_hier.edges()))

Number of nodes: 69

Number of edges: 69


In [27]:
# add node attributes to dataframe for easier access
hier_df = pd.DataFrame.from_dict(dict(G_hier.nodes(data=True)), orient='index')
hier_df['system_ID']=hier_df.index.tolist()
# some columns are not the right type
hier_df['CD_MemberList_Size']=[int(x) for x in hier_df['CD_MemberList_Size'].tolist()]
hier_df['HiDeF_persistence']=[int(x) for x in hier_df['HiDeF_persistence'].tolist()]
hier_df.head()

,CD_MemberList,CD_MemberList_Size,CD_Labeled,CD_MemberList_LogSize,CD_CommunityName,CD_AnnotatedMembers,CD_AnnotatedMembers_Size,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,HiDeF_persistence,represents,name,system_ID
0,AIMP1 SUOX ETFB CLPTM1 GRHPR EGLN3 FARSB KCND1...,747,False,9.545,,,0,0.0,0.0,122,C748,C748,0
1,CHKB SLC44A2 SLC44A3 GPD1L SLC44A1 SLC44A5 GPD1,7,False,2.807,,,0,0.0,0.0,26,C768,C768,1
2,M6PR SYNJ1 NLRP3 DNAJC6 SCARB2 DNASE2 INPP4A,7,False,2.807,,,0,0.0,0.0,18,C771,C771,2
3,KIF1A TCP11X2 PLA2G6 TUBA4B TUBB4A TUBG2 TBCD,7,False,2.807,,,0,0.0,0.0,6,C773,C773,3
4,EPM2A GYG1 PPP1R3C NHLRC1 GYG2 AGL,6,False,2.585,,,0,0.0,0.0,56,C775,C775,4


12. Remove systems with no seed genes (OPTIONAL)

In [28]:
hier_df.index=hier_df['name']
hier_df.head()

num_d1_seeds = []
frac_d1_seeds=[]

systems_keep = []
for c in hier_df.index.tolist():
    system_genes = hier_df['CD_MemberList'].loc[c].split(' ')
    d1_temp = list(np.intersect1d(system_genes,D1_genes))

    num_d1_temp = len(d1_temp)
    if (num_d1_temp)>0: # keep the system if it has at least 1 seed gene
        systems_keep.append(c)
        num_d1_seeds.append(num_d1_temp)
        
        frac_d1_seeds.append((num_d1_temp)/float(len(system_genes)))

        
frac_no_seeds = np.subtract(1.0,np.array([frac_d1_seeds]).sum(axis=0))

hier_df = hier_df.loc[systems_keep]
hier_df['num_d1_seeds']=num_d1_seeds
hier_df['frac_d1_seeds']=frac_d1_seeds
hier_df['frac_no_seeds']=frac_no_seeds
print("Number of nodes with seed genes:", len(hier_df))

hier_df.head()
    

Number of nodes with seed genes: 59


,CD_MemberList,CD_MemberList_Size,CD_Labeled,CD_MemberList_LogSize,CD_CommunityName,CD_AnnotatedMembers,CD_AnnotatedMembers_Size,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,HiDeF_persistence,represents,name,system_ID,num_d1_seeds,frac_d1_seeds,frac_no_seeds
name,,,,,,,,,,,,,,,,
C748,AIMP1 SUOX ETFB CLPTM1 GRHPR EGLN3 FARSB KCND1...,747,False,9.545,,,0,0.0,0.0,122,C748,C748,0,196,0.262383,0.737617
C768,CHKB SLC44A2 SLC44A3 GPD1L SLC44A1 SLC44A5 GPD1,7,False,2.807,,,0,0.0,0.0,26,C768,C768,1,1,0.142857,0.857143
C771,M6PR SYNJ1 NLRP3 DNAJC6 SCARB2 DNASE2 INPP4A,7,False,2.807,,,0,0.0,0.0,18,C771,C771,2,3,0.428571,0.571429
C773,KIF1A TCP11X2 PLA2G6 TUBA4B TUBB4A TUBG2 TBCD,7,False,2.807,,,0,0.0,0.0,6,C773,C773,3,4,0.571429,0.428571
C775,EPM2A GYG1 PPP1R3C NHLRC1 GYG2 AGL,6,False,2.585,,,0,0.0,0.0,56,C775,C775,4,2,0.333333,0.666667


In [29]:
# prune G_hier--> only keep systems with at least one seed gene

nkeep=[]
for n in list(G_hier.nodes()):
    if G_hier.nodes(data=True)[n]['name'] in systems_keep:
        nkeep.append(n)
        

G_hier = nx.subgraph(G_hier, nkeep)
print("Number of nodes with seed genes:", len(G_hier.nodes()))
print("Number of edges remaining:", len(G_hier.edges()))

Number of nodes with seed genes: 59
Number of edges remaining: 59


In [30]:
network_colocalization.view_G_hier(G_hier)

CytoscapeWidget(cytoscape_layout={'name': 'cose'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…

14. Annotate systems with gprofiler.
Annotate moderately sized systems (between 50 to 1000 genes per system) if they are significantly enriched for a Gene Ontology biological process. Also require that the GO term is enriched with 
 and shares at least 3 genes with the system to annotate, to increase the stringency of the annotation. Label the system using the GO term that meets these criteria, and has the highest sum of precision and recall. Systems which have no GO terms meeting these criteria are labeled with their unique system ID.

In [31]:
# gprofiler annotation of clusters

system_name_list = []
for p in hier_df.index.tolist():
    focal_genes=hier_df['CD_MemberList'].loc[p].split(' ')
    print(p)
    print(len(focal_genes))
    if len(focal_genes)>2:
        gp_temp = pd.DataFrame(gp.profile(focal_genes,significance_threshold_method='fdr',
                                               sources=['REAC']))
        if len(gp_temp)>0: # make sure data is not empty
            
            # make sure terms are specific, and overlap with at least 3 genes
            gp_temp = gp_temp[(gp_temp['term_size']<1000)&(gp_temp['term_size']>50)]
            gp_temp = gp_temp[gp_temp['intersection_size']>=3]
            
            gp_temp = gp_temp[gp_temp['p_value']<1E-5] # set a stringent pvalue threshold
            
            gp_temp = gp_temp.sort_values('recall',ascending=False)
            
            if len(gp_temp)>1:
                system_name_list.append(gp_temp.head(1)['name'].tolist()[0])
            else:
                system_name_list.append(p)
        else:
            system_name_list.append(p)
            

        display(gp_temp.head())
        
    else:
        system_name_list.append(p)

C748
747


,description,effective_domain_size,intersection_size,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size
5,Pyruvate metabolism and Citric Acid (TCA) cycle,10842,26,Pyruvate metabolism and Citric Acid (TCA) cycle,REAC:R-HSA-71406,1.162152e-17,[REAC:R-HSA-1428517],0.047016,query_1,553,0.481481,True,REAC,54
12,Peroxisomal protein import,10842,25,Peroxisomal protein import,REAC:R-HSA-9033241,2.169400e-15,[REAC:R-HSA-9609507],0.045208,query_1,553,0.416667,True,REAC,60
9,Sphingolipid metabolism,10842,31,Sphingolipid metabolism,REAC:R-HSA-428157,6.387769e-17,[REAC:R-HSA-556833],0.056058,query_1,553,0.360465,True,REAC,86
1,The citric acid (TCA) cycle and respiratory el...,10842,59,The citric acid (TCA) cycle and respiratory el...,REAC:R-HSA-1428517,3.049943e-30,[REAC:R-HSA-1430728],0.106691,query_1,553,0.333333,True,REAC,177
6,Metabolism of water-soluble vitamins and cofac...,10842,37,Metabolism of water-soluble vitamins and cofac...,REAC:R-HSA-196849,2.183299e-17,[REAC:R-HSA-196854],0.066908,query_1,553,0.305785,True,REAC,121


C768
7


,description,effective_domain_size,intersection_size,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size
0,Glycerophospholipid biosynthesis,10842,7,Glycerophospholipid biosynthesis,REAC:R-HSA-1483206,4.336870e-13,[REAC:R-HSA-1483257],1.000000,query_1,7,0.054688,True,REAC,128
4,"Transport of bile salts and organic acids, met...",10842,4,"Transport of bile salts and organic acids, met...",REAC:R-HSA-425366,3.689452e-07,[REAC:R-HSA-425407],0.571429,query_1,7,0.047619,True,REAC,84
1,Phospholipid metabolism,10842,7,Phospholipid metabolism,REAC:R-HSA-1483257,7.409963e-12,[REAC:R-HSA-556833],1.000000,query_1,7,0.033333,True,REAC,210
3,Metabolism of lipids,10842,7,Metabolism of lipids,REAC:R-HSA-556833,2.490100e-08,[REAC:R-HSA-1430728],1.000000,query_1,7,0.009563,True,REAC,732


C771
7


...

,description,effective_domain_size,intersection_size,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size
1,Metabolism of amino acids and derivatives,10842,4,Metabolism of amino acids and derivatives,REAC:R-HSA-71291,0.000003,[REAC:R-HSA-1430728],1.0,query_1,4,0.010811,True,REAC,370


C799
4


,description,effective_domain_size,intersection_size,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size


C816
4


,description,effective_domain_size,intersection_size,name,native,p_value,parents,precision,query,query_size,recall,significant,source,term_size
